## Initialize the environment

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('Occupancy_Estimation.csv')

## Question 1: Proportion Test for Sound Spikes

**Goal:** 95% CI for $p=\Pr[S1\_Sound>1]$, test $H_0:p=0.025$.

1. **Count spikes**:

In [9]:
spikes = (df["S1_Sound"] > 1).sum()
n = len(df)
phat = spikes / n

print(f"Total samples: {n}")
print(f"Spikes: {spikes}")
print(f"Sample proportion (p̂): {phat:.5f}")

Total samples: 10129
Spikes: 280
Sample proportion (p̂): 0.02764


2. **Check normal approx**: ensure $n p_0\ge10$ and $n(1-p_0)\ge10$.

In [10]:
p0 = 0.025
np0 = n * p0
nq0 = n * (1 - p0)
print(f"np0 = {np0}, n(1-p0) = {nq0}")
print("Normal approximation is valid." if np0 >= 10 and nq0 >= 10 else "Normal approximation is NOT valid.")

np0 = 253.22500000000002, n(1-p0) = 9875.775
Normal approximation is valid.


3. **95% CI** (Wald):

   $$
     \hat p \pm z_{0.975} \sqrt{\frac{\hat p(1-\hat p)}{n}}
   $$

In [11]:
import numpy as np
from scipy.stats import norm

z = norm.ppf(0.975)  # 95% confidence
se = np.sqrt(phat * (1 - phat) / n)
margin = z * se
ci_lower = phat - margin
ci_upper = phat + margin

print(f"95% CI: [{ci_lower:.5f}, {ci_upper:.5f}]")

95% CI: [0.02445, 0.03084]


4. **Z-test**:

   $$
     z = \frac{\hat p - 0.025}{\sqrt{0.025\,(1-0.025)/n}},\quad
     p\text{-value}=2(1-\Phi(|z|))
   $$

In [12]:
# Z-test
se_h0 = np.sqrt(p0 * (1 - p0) / n)
z_stat = (phat - p0) / se_h0
p_value = 2 * (1 - norm.cdf(abs(z_stat)))

print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Reject H0: There is a significant difference from 0.025.")
else:
    print("Fail to reject H0: No significant difference from 0.025.")

Z-statistic: 1.7040
P-value: 0.0884
Fail to reject H0: No significant difference from 0.025.


### Interpretation

* If $p$-value < 0.05, conclude $p\neq0.025$.
* Report CI, e.g. “We estimate $p=…$ with 95% CI $[…,…]$.”
* Confirm normal-approx holds (both $np_0$ and $n(1-p_0)\ge10$).

## Question 2: One-Sample Mean Test for Temperature

**Goal:** Is $\mu_{\rm temp}=25$°C?

1. **Compute sample mean & std**: